In [75]:
"""
Author(s): Yuqin Cui
Created: 2019-05-08
Edited: 2019-06-10
"""

'\nAuthor(s): Yuqin Cui\nCreated: 2019-05-08\nEdited: 2019-06-10\n'

In [76]:
import numpy as np
import pandas as pd
import panel as pn
import param
import hvplot.pandas
import holoviews as hv
from holoviews import opts, streams
from bokeh.models import CustomJS, BoxSelectTool, ColumnDataSource
from colorcet import palette
from bokeh.models.widgets.buttons import Button


from holoviews.streams import Selection1D


In [77]:
fname = 'GephiMatrix_author_similarity.csv'
f = open(fname, 'r')

# Get author names
line1 = f.readline()
names = line1[1:].split(';');

seen = {}
dupes = []

for index, name in enumerate(names):
    if name not in seen:
        seen[name] = 1
    else:
        if seen[name] == 1:
            dupes.append((index, name))
        seen[name] += 1

# add 1, 2 etc after the name
for pair in dupes:
    index = pair[0]
    name = pair[1]
    for i in range(seen[name]):
        names[index] = name + str((i+1))
        #print(names[index])

# Read csv
df = pd.read_csv(f, names=names, sep=';')

# Fix it
df = df.reset_index(level=1)
names.append("delete")
names = [name.replace('_', ' ') for name in names]
df.columns = names
del df["delete"]
df.set_index([df.columns], inplace=True)

# Get names again for later use
names = df.columns.tolist()

# Get 150*150 sub matrix since otherwise the plot is very slow..
#df = df.head(1000)[names[0:1000]]
names = df.columns.tolist()

df_original = df.copy()


df_node_number = len(df.columns)

In [78]:
df_node_number

1053

In [79]:
from sklearn.cluster import KMeans
import numpy as np
import numpy as np
import collections 

In [80]:
def K_mean_cluster(dataframe):
    node_number = len(dataframe.columns)
    if node_number <= 170:
        print("There is no need to cluster")
    elif node_number >= 1500:
        print("The dataset is too large to cluster, use edge weight filter")
    else:
        X = np.array(dataframe.values)

        n_clusters= node_number//170

        kmeans = KMeans(n_clusters= node_number//170, random_state=0).fit(X)
        cluster_label = kmeans.labels_
        cluster_label = cluster_label.tolist()
        counter=collections.Counter(cluster_label)

        names = dataframe.columns.tolist()
        cluster_label = pd.Series(cluster_label)
        names_series = pd.Series(names)
        cluster_label_df = pd.concat([names_series, cluster_label], axis=1)
        cluster_label_df = pd.DataFrame({'index': names,'cluster_label': cluster_label})

    #     print(counter)
    #     print(counter.values())
    #     print(counter.keys())
    #     print(counter.most_common(1))
        check_frequency = counter.most_common(1)
        largest_cluster_label = check_frequency[0][0]
        largest_cluster_node_number = check_frequency[0][1]

        #check whether it's extremely unevenly distributed
        if largest_cluster_node_number >= node_number//3 and largest_cluster_node_number >=200:
            #sparate the whole dataset into 2 datasets, one biggest, one combined small others 
            cluster_n = 2
#             if cluster_label_df['cluster_label'] != largest_cluster_label:
#                 cluster_label_df['cluster_label'] = largest_cluster_label + 1 
            cluster_label_df.loc[cluster_label_df['cluster_label'] != largest_cluster_label, ['cluster_label']] = largest_cluster_label+1
            new_counter = {largest_cluster_label: largest_cluster_node_number, largest_cluster_label + 1 : node_number - largest_cluster_node_number}
            
            #
#             cluster_selected_index = cluster_label_df.loc[cluster_label_df['cluster_label'] == largest_cluster_label]
#             cluster_selected_name_series = cluster_selected_index['index']
#             cluster_selected_df = df.loc[cluster_selected_name_series, cluster_selected_name_series]
#             cluster_n, new_counter, cluster_label_df = K_mean_cluster(cluster_selected_df)
            #
            
            return cluster_n, new_counter, cluster_label_df
        else:
            cluster_n = n_clusters
            #print(counter.keys(), counter.values())
            return cluster_n, counter, cluster_label_df

In [81]:
cluster_n, new_counter, new_cluster_label_df = K_mean_cluster(df)
cluster_n, new_counter, new_cluster_label_df

(6,
 Counter({5: 187, 3: 196, 0: 214, 2: 230, 4: 91, 1: 135}),
                          index  cluster_label
 0                   Jim Thomas              5
 1       Eleftherios Koutsofios              3
 2             Lawrence A. Rowe              0
 3          Jonathan I. Helfman              5
 4                Mary Brewster              2
 5                     Min Chen              4
 6        Christopher G. Healey              2
 7                   Mac Holden              4
 8                      M. Gray              2
 9                Jason Stewart              1
 10            Volker Vogelmann              2
 11        Benjamin B. Bederson              2
 12             Maureen Tingley              2
 13                  C. K. Yung              5
 14            S. Joy Mountford              4
 15               L. D. Bergman              5
 16             Rich Gossweiler              0
 17                 M. Maruyama              2
 18           David P. Helmbold             

In [82]:
from bokeh.plotting import figure,show
from bokeh.layouts import row
from bokeh.models import CustomJS, ColumnDataSource
from bokeh.plotting import figure, output_file, show


#output_file("title.html")
# TOOLTIPS = [
#     ("index", "$index"),
#     ("(x,y)", "($x, $y)"),
#     ("desc", "@desc"),
# ]

#p = figure(plot_width=400, plot_height=400, title="Cluster Size",tooltips=TOOLTIPS)
p = figure(plot_width=400, plot_height=400, title="Cluster Size", tools=['hover', 'tap'])
p.title.text_color = "olive"
p.title.text_font = "times"
p.title.text_font_style = "italic"

cluster_number, counter, cluster_label_df = K_mean_cluster(df)






for i,j in counter.items():
    #print(i,j)
    if i%2 == 0:
        p.circle(int(i), int(i), size = j/10, color="olive", alpha=0.5)
    else:
        p.circle(int(i), int(i), size = j/10, color="navy", alpha=0.5)
        
show(p)

In [83]:
#------------------------------------------------------------------------------------------------------#

In [84]:
data = ColumnDataSource({"Label": list(counter.keys()), "Size": list(counter.values()),
                         "circle_size": [item/10 for item in counter.values()],
                         "color": [["olive","navy"][i%2] for i in counter.keys()]})

data.selected.js_on_change('indices', CustomJS(args=dict(data=data), code="""
        let inds = cb_obj.indices;
        let labels = []
        for (let i = 0; i < inds.length; i++){
            let label = data.data.Label[inds[i]]
            labels[i] = label
        }
        console.log(labels)
        // $.post("/handle-cluster-callback", {'labels': labels})
    """))

TOOLTIPS = [
    ("Label", "@Label"),
    ("Number of nodes in this cluster", "@Size"),
]

p = figure(plot_width=400, plot_height=400, title="Choose your cluster", tools=['hover', 'tap', 'lasso_select', 'reset'], tooltips=TOOLTIPS)
p.circle("Label", "Label", size="circle_size", color="color", alpha=0.5, source=data)
p.title.text_color = "olive"
p.title.text_font = "times"
p.title.text_font_style = "italic"

p.background_fill_color = None
p.border_fill_color = None
p.outline_line_color = None
#p.xaxis.visible = False
p.yaxis.visible = False
p.grid.visible = False
show(p)

In [85]:
#------------------------------------------------------------------------------------------------------#

In [86]:
print(cluster_label_df)

                         index  cluster_label
0                   Jim Thomas              5
1       Eleftherios Koutsofios              3
2             Lawrence A. Rowe              0
3          Jonathan I. Helfman              5
4                Mary Brewster              2
5                     Min Chen              4
6        Christopher G. Healey              2
7                   Mac Holden              4
8                      M. Gray              2
9                Jason Stewart              1
10            Volker Vogelmann              2
11        Benjamin B. Bederson              2
12             Maureen Tingley              2
13                  C. K. Yung              5
14            S. Joy Mountford              4
15               L. D. Bergman              5
16             Rich Gossweiler              0
17                 M. Maruyama              2
18           David P. Helmbold              4
19                  C. Dunmire              0
20               C. C. Gomberg    

In [62]:
def get_dataframe_from_dot(dataframe, cluster_umber):
    cluster_n, counter, cluster_label_df = K_mean_cluster(dataframe)
    cluster_selected_index = cluster_label_df.loc[cluster_label_df['cluster_label'] == cluster_umber]
    cluster_selected_name_series = cluster_selected_index['index']
    cluster_selected_df = df.loc[cluster_selected_name_series, cluster_selected_name_series]
    return cluster_selected_df

In [63]:
get_dataframe_from_dot(df, 3)

,Jason Stewart,Robert Cohen,BenoÃ®t Hudson,Scott A. Guyer,Robin Lampert,Shari Lawrence Pfleeger,Thomas H. Cormen,Ravinder Bhogal,Larry Dane Brimner,Bruce Hendrickson,...,Michael S. Miller,Yigal Arens,T. A. DeFanti,Kaushai Dalal,M. D. Brown,Maria Shneerson,Scott Meyers,Przemyslaw Prusinkiewicz,Chris Greenhalgh,Ken Pier
Jason Stewart,1.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
Robert Cohen,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
BenoÃ®t Hudson,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
Scott A. Guyer,0.0,0.0,0.0,1.000000,0.57735,0.408248,0.408248,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.577350,0.000000,0.000000,0.408248,0.000000,0.000000,0.0
Robin Lampert,0.0,0.0,0.0,0.577350,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
Shari Lawrence Pfleeger,0.0,0.0,0.0,0.408248,0.00000,1.000000,0.500000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.707107,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
Thomas H. Cormen,0.0,0.0,0.0,0.408248,0.00000,0.500000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.707107,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
Ravinder Bhogal,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
Larry Dane Brimner,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0
Bruce Hendrickson,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
